In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
from os import path

# Parsing for Data Download Links

In [ ]:
with open("/content/drive/MyDrive/cse6242/historical_data_buoy.html", 'r') as file:
    html_content = file.read()

In [ ]:
# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Define the regular expression for each type of data
pattern_section = re.compile(r'<ul class="histfiles">[^<]*</ul>')
# Find all <ul> tags that match the pattern
matching_ul_tags = soup.find_all(lambda tag: tag.name == 'ul' )

sections = []
station_ids = set()
for ul_tag in matching_ul_tags:
    # Define the regular expression pattern
    pattern = re.compile(r'<li>[0-9a-zA-Z]+:[^<]*</li>') #

    # Find all <li> tags that match the pattern
    matching_li_tags = ul_tag.find_all(lambda tag: tag.name == 'li' )
    #print(matching_li_tags)

    # Extract links associated with k-largest year numbers
    k = 3  # Adjust k as needed

    station_link = {} #dictionary that stores the station id as key and list of links as values

    for li_tag in matching_li_tags:

        # Extract the <a> tags within the <li> tag
        a_tags = li_tag.find_all('a')

        # Extract the href attributes from the <a> tags
        hrefs = [a.get('href') for a in a_tags]

        station_id = str(li_tag)[4:9]
        station_ids.add(station_id)
        # Get the link associated with the k-largest year number
        if len(hrefs) >= k:
            #k_largest_link = hrefs[k - 1]
            station_link[station_id] = hrefs[-k:]

    sections.append(station_link)

In [ ]:
print(station_ids)

{'42053', '42038', 'APAM2', 'EB43:', '45205', '51211', 'PTRP4', 'MISM1', 'LNEL1', 'MDKF1', 'CRGA2', 'SKCF1', 'OPTF1', 'BSBM4', '42925', 'NWPO3', '42085', '44073', 'CNII2', '41051', '46111', 'SLVM5', '41046', '46223', '41115', 'WIWF1', 'VBBA3', 'CNDO1', '46250', 'SISW1', 'PORT2', 'LPWA2', 'FPTM4', 'ILDL1', 'FSTI2', '45207', '42922', '46001', 'CPVM2', 'LBSF1', 'PNLM4', 'ACMN4', 'NPDW3', 'SCXA2', '45029', 'FRXM3', 'MISMA', 'THRO1', 'VERV4', 'BHRI3', '41030', 'BLTA2', 'PXOC1', '44028', 'KDAA2', '45015', 'HPLM2', '44091', 'MRHO1', 'FRKF1', 'PCXA2', '45201', '46216', 'OCIM2', '42931', 'PTRF1', 'MCYI3', '42093', '46415', 'BMTW1', '45204', '46220', 'ICYA2', 'DKKF1', '32404', '44402', 'EREP1', '42004', 'AKXA2', 'ABAN6', '46243', '32ST2', '42886', 'PSTN6', 'TBIM4', 'APNM4', 'JMPN7', '42361', 'D065W', '44008', 'WRXA2', '44006', 'MRKA2', '55016', 'SECG1', 'FCGT2', 'LSCM4', 'DUKN7', '46017', '51101', '44235', 'MPCL1', 'BLTM2', '51203', 'DSLN7', '41040', '46013', '46265', '52403', 'RCYF1', '46234', 

In [ ]:
sections[1]

{'<a id': ['/download_data.php?filename=venf1l2006.txt.gz&dir=data/historical/wlevel/',
  '/download_data.php?filename=venf1l2007.txt.gz&dir=data/historical/wlevel/',
  '/download_data.php?filename=venf1l2008.txt.gz&dir=data/historical/wlevel/'],
 '0Y2W3': ['/download_data.php?filename=0y2w3h2013.txt.gz&dir=data/historical/stdmet/',
  '/download_data.php?filename=0y2w3h2014.txt.gz&dir=data/historical/stdmet/',
  '/download_data.php?filename=0y2w3h2015.txt.gz&dir=data/historical/stdmet/'],
 '23020': ['/download_data.php?filename=23020h2008.txt.gz&dir=data/historical/stdmet/',
  '/download_data.php?filename=23020h2009.txt.gz&dir=data/historical/stdmet/',
  '/download_data.php?filename=23020h2010.txt.gz&dir=data/historical/stdmet/'],
 '32012': ['/download_data.php?filename=32012k2016.txt.gz&dir=data/historical/swr2/',
  '/download_data.php?filename=32012k2017.txt.gz&dir=data/historical/swr2/',
  '/download_data.php?filename=32012k2018.txt.gz&dir=data/historical/swr2/'],
 '32301': ['/downl

In [ ]:
station_ids = list(station_ids)
type(station_ids)

list

# Scraping for station coordinates

Sample station page: https://www.ndbc.noaa.gov/station_page.php?station=21415
where station id is the last 5 digits/letters

Plan:
- get a set of all stations ids in the previously complied sections
- append the station id to get the link for each station
- scrape out the cooridinates
- store the key (station id) - value (coordinates pair) in a dictionary.

In [ ]:
url_base ="https://www.ndbc.noaa.gov/station_page.php?station="

In [ ]:
station_coord = {}
failed_stations = []
for station_id in station_ids:
    if station_id[-1] == ":":
      station_id = station_id[:-1]
    url = f"https://www.ndbc.noaa.gov/station_page.php?station={station_id}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the <link> element with the specified attributes
        link_tag = soup.find('link', {'rel': 'alternate', 'type': 'application/rss+xml', 'href': re.compile(r'^/rss/ndbc_obs_search.php')})

        # Extract the title attribute value
        if link_tag and 'title' in link_tag.attrs:
            title_value = link_tag['title']
            # Extract coordinates from the title using a regular expression
            coordinates_match = re.search(r'near ([0-9.]+[A-Z]{1}) ([0-9.]+[A-Z]{1})', title_value)
            if coordinates_match:
                latitude = coordinates_match.group(1)
                longitude = coordinates_match.group(2)

                # Print the coordinates
                #print(f"Coordinates: {latitude} {longitude}")
                station_coord[station_id] = [latitude, longitude]
            else:
                station_coord[station_id] = [-1, -1] #null
        else:
            print("No matching <link> element found")
    else:
        failed_stations.append(station_id)
        #print(f"Failed to retrieve data for Station {station_id}. Status code: {response.status_code}")

In [ ]:
len(station_coord)

1372

In [ ]:
failed_stations_save = pd.Series(failed_stations)
path = "/content/drive/MyDrive/cse6242/failed_stations.csv"
failed_stations_save.to_csv(path, index=False)

In [ ]:
station_coord_dict = station_coord.to_dict()

In [ ]:
station_id_keys = station_coord_dict.keys()
print(type(station_id_keys))
station_coord_vals = station_coord_dict.values()
station_coord_new = pd.DataFrame(station_id_keys)
station_coord_new["coordinates"] = station_coord_vals
station_coord_new

<class 'dict_keys'>


,0,coordinates
0,EB31,"{0: '27.000N', 1: '86.000W'}"
1,MEYC1,"{0: '36.605N', 1: '121.889W'}"
2,EGKF1,"{0: '27.601N', 1: '82.759W'}"
3,46255,"{0: '33.400N', 1: '119.651W'}"
4,EMAT2,"{0: '28.710N', 1: '95.914W'}"
...,...,...
1367,45199,"{0: '42.702N', 1: '87.647W'}"
1368,PTJN6,"{0: '40.948N', 1: '73.070W'}"
1369,46116,"{0: '46.287N', 1: '124.016W'}"
1370,EB01,"{0: '36.500N', 1: '73.500W'}"


In [ ]:
path = "/content/drive/MyDrive/cse6242/station_coord.csv"
station_coord_new.to_csv(path, index=False)

# Get Files from Links!

Plan:
- for each section:
  - if station has coordinate,
    - download the 3 historical data file associated with the station id
    - put it into the folder of the section and then the folder for the station

In [ ]:
import os.path
from os import path

In [ ]:
failed_stations = pd.read_csv('/content/drive/MyDrive/cse6242/failed_stations.csv')

In [ ]:
download_base = "https://www.ndbc.noaa.gov/view_text_file.php?filename=0y2w3h2012.txt.gz&dir=data/historical/stdmet/"

In [ ]:
sections_name = {"stdmet": "h", "cwind": "c", "adcp": "a", "ocean": "o", "swden": "w", "swdir": "d", "swdir2": "i", "swr1": "j", "swr2": "k",  "wlevel": "l", "dart": "t", }

In [ ]:
important_section_names = { "ocean": "o", "stdmet": "h"}
stdmet_vals = ["YYYY", "MM", "DD", "hh", "mm", "WVHT", "DPD"] #significant wave height, dominant wave period
ocean_vals = ["YYYY", "MM", "DD", "hh", "mm", "OTMP","SAL"] # ocean temperature, salinity

In [ ]:
#Code for looping over all "sections" 11-30
no_data_stations = []
for station_id in sections[1]: #loop through every station in the standard meteorlogical section
    if station_id not in failed_stations:
      for download_link in sections[1][station_id]:
          if (station_id in sections[7].keys()): #if we also have the oceanographic data for this station
            match = re.search(r'h(\d+)\.txt', download_link) #find year in link
            section = download_link.rsplit('/', 2)[-2]
            keyword = sections_name[section]
            # Extract the matched elements

            if match:
                year = match.group(1)#get year

                #create dir if doesnt exist already
                if path.exists(f'/content/drive/MyDrive/cse6242/{section}') == False:
                    os.mkdir(f'/content/drive/MyDrive/cse6242/{section}')

                if path.exists(f'/content/drive/MyDrive/cse6242/{section}/{station_id}') == False:
                    os.mkdir(f'/content/drive/MyDrive/cse6242/{section}/{station_id}')


                    #https://www.ndbc.noaa.gov/view_text_file.php?filename=0y2w3h2013.txt.gz&dir=data/historical/stdmet/
                link_stdmet =  f"https://www.ndbc.noaa.gov/view_text_file.php?filename={station_id.lower()}{keyword}{year}.txt.gz&dir=data/historical/{section}/"
                #print(link)
                try:
                    # get standard meteological data
                    data_stdmet = pd.read_csv(link_stdmet, delim_whitespace=True, usecols=stdmet_vals, error_bad_lines=False)
                    save_path = f"/content/drive/MyDrive/cse6242/wave_data/11-30/{section}/{station_id}/{year}.csv"
                    data_stdmet.to_csv(save_path, index=False)

                except pd.errors.EmptyDataError:
                    # Handle the case where the link contains no data
                    print(f"No data found in the provided link for station{station_id} at year {year}")
                    no_data_stations.append(station_id)


In [ ]:
for section_i in range(len(sections)):

    for station_id in sections[section_i]:
        if station_id not in failed_stations:
          for download_link in sections[section_i][station_id]:
              match = re.search(r'h(\d+)\.txt', download_link) #get year
              section = download_link.rsplit('/', 2)[-2]

              #check if the section is what we are interested in
              if section in important_section_names.keys():
                keyword = sections_name[section]

                if section == "stdmet":
                  selected_columns = stdmet_vals
                elif section == "ocean":
                  selected_columns = ocean_vals

                # Extract the matched elements
                if match:
                    year = match.group(1)

                    #create dir if doesnt exist already
                    if path.exists(f'/content/drive/MyDrive/cse6242/{section}') == False:
                        os.mkdir(f'/content/drive/MyDrive/cse6242/{section}')

                    if path.exists(f'/content/drive/MyDrive/cse6242/{section}/{station_id}') == False:
                        os.mkdir(f'/content/drive/MyDrive/cse6242/{section}/{station_id}')


                    #https://www.ndbc.noaa.gov/view_text_file.php?filename=0y2w3h2013.txt.gz&dir=data/historical/stdmet/
                    link =  f"https://www.ndbc.noaa.gov/view_text_file.php?filename={station_id.lower()}{keyword}{year}.txt.gz&dir=data/historical/{section}/"
                    #print(link)
                    try:

                        data = pd.read_csv(link, delim_whitespace=True, usecols=selected_columns, error_bad_lines=False)
                        save_path = f"/content/drive/MyDrive/cse6242/{section}/{station_id}/{year}.csv"
                        data.to_csv(save_path, index=False)

                    except pd.errors.EmptyDataError:
                        # Handle the case where the link contains no data
                        print(f"No data found in the provided link for station{station_id} at year {year}")
                        no_data_stations.append(station_id)
              else:
                  pass




# Section for Ocean + Stdmet data

- plan:
  - for a station id in in standard meteorology section
    - find all of the available years
    - find all of the available years in oceanography
    - find the intersection of these sets
    - get the k largest years

In [ ]:
# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Define the regular expression for each type of data
pattern_section = re.compile(r'<ul class="histfiles">[^<]*</ul>')
# Find all <ul> tags that match the pattern
matching_ul_tags = soup.find_all(lambda tag: tag.name == 'ul' )

sections = []
station_ids = set()
for ul_tag in matching_ul_tags:
    # Define the regular expression pattern
    pattern = re.compile(r'<li>[0-9a-zA-Z]+:[^<]*</li>') #

    # Find all <li> tags that match the pattern
    matching_li_tags = ul_tag.find_all(lambda tag: tag.name == 'li' )

    # Extract links associated with k-largest year numbers
    k = 3  # Adjust k as needed

    station_link = {} #dictionary that stores the station id as key and list of links as values

    for li_tag in matching_li_tags:

        # Extract the <a> tags within the <li> tag
        a_tags = li_tag.find_all('a')

        # Extract the href attributes from the <a> tags
        hrefs = [a.get('href') for a in a_tags]
        # years = []
        # print(hrefs)
        # for href in hrefs:
        #   print(href, "---")
        #   match = re.search(r'h(\d+)\.txt', href) #get year
        #   if match:
        #     year = match.group(1)
        #     years.append(year)
        station_id = str(li_tag)[4:9]
        station_ids.add(station_id)
        # Get the link associated with the k-largest year number
        if len(hrefs) >= k:
            #k_largest_link = hrefs[k - 1]
            station_link[station_id] =  hrefs

    sections.append(station_link)

In [ ]:
sections[1] # 1: stdmet, 7: ocean

In [ ]:

sections[2].get("42067")

- for station_id in stdmet section, get val corresponding to key
- check if station_id also in sections[7] ocean
  - if yes
    - get the years for ocean
    - set both to sets and intersect the sets
    - pick the top k years
    - get the datasets

In [ ]:
important_section_names = { "ocean": "o", "stdmet": "h"}
stdmet_vals = [ "MM", "DD", "hh", "mm", "WVHT", "DPD"] #significant wave height, dominant wave period
ocean_vals = [ "MM", "DD", "hh", "mm", "OTMP","SAL"] # ocean temperature, salinity

In [ ]:
combined_dict = {**sections[1], **sections[2]}
print(combined_dict)

{'<a id': [None, '/faq/measdes.shtml#wlevel', '/download_data.php?filename=46050l1997.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=dpia1l1996.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=dpia1l1997.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=dpia1l1998.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=dpia1l2002.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=dpia1l2003.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=dpia1l2004.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=dpia1l2005.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=gdil1l1996.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=gdil1l1997.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=gdil1l1998.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=gdil1l1999.txt.gz&dir=data/historical/wlevel/', '/download_data.php?filename=g

In [ ]:
ocean_stations = list(sections[7].keys())
stdmet_stations = list(sections[1].keys()) + list(sections[2].keys())
print( stdmet_stations)

['<a id', '0Y2W3', '23020', '32012', '32301', '32302', '32487', '32ST1', '32ST2', '41001', '41002', '41003', '41004', '41005', '41006', '41008', '41009', '41010', '41012', '41013', '41016', '41018', '41021', '41024', '41025', '41027', '41029', '41030', '41033', '41035', '41036', '41037', '41038', '41040', '41041', '41043', '41044', '41046', '41047', '41048', '41049', '41051', '41052', '41053', '41056', '41057', '41060', '41062', '41063', '41064', '41108', '41109', '41110', '41112', '41113', '41114', '41115', '41117', '41140', '41159', '41193', '41NT1', '42001', '42002', '42003', '42005', '42007', '42008', '42009', '42011', '42012', '42013', '42014', '42015', '42016', '42019', '42020', '42021', '42022', '42023', '42025', '42026', '42035', '42036', '42038', '42039', '42040', '42041', '42043', '42044', '42045', '42046', '42047', '42055', '42056', '42057', '42058', '42059', '42060', '42067', '42085', '42089', '42095', '42097', '42098', '42099', '42360', '42361', '42362', '42363', '42364', 

In [ ]:
i = 0
success_stations = []
failed_year_ocean = []
for station_id in stdmet_stations:
  station_id_2 = station_id
  if station_id_2 in ocean_stations:
    stdmet_years = []
    ocean_years = []

    #find available years in standard meteo
    #print(sections_1[station_id])
    if sections[2].get(station_id) is not None:
      for download_link in sections[2].get(station_id):
        section = download_link.rsplit('/', 2)[-2]
        match = re.search(r'h(\d+)\.txt', download_link) #get year
        if match:
          year = match.group(1)
          stdmet_years.append(year)
    else:
      print(f"No download links found for station_id: {station_id}")

    if sections[7].get(station_id) is not None:
      #find available years in ocean
      for download_link in sections[7][station_id_2]:
        print(download_link)
        section = download_link.rsplit('/', 2)[-2]
        match = re.search(r'o(\d+)\.txt', download_link) #get year
        if match:
          year = match.group(1)
          ocean_years.append(year)
    else:
      print(f"No download links found for station_id: {station_id}")
    # print(ocean_years)
    # print(stdmet_years)
    ocean_set = set(ocean_years)
    stdmet_set = set(stdmet_years)
    # Find the intersection of the two sets
    intersection_set = ocean_set.intersection(stdmet_set)

    # Convert the intersection set to a list
    intersection_list = list(intersection_set)

    # Sort the intersection list in descending order
    intersection_list.sort(reverse=True)

    # Return the k largest values from the intersection list
    k_largest_values = intersection_list[:2]

    print(intersection_list)
    for section in ["stdmet", "ocean"]:
      if section == "stdmet":
        selected_columns = stdmet_vals
      elif section == "ocean":
        selected_columns = ocean_vals

      for year in intersection_list:
        #create dir if doesnt exist already
        if path.exists(f'/content/drive/MyDrive/cse6242/{section}') == False:
            os.mkdir(f'/content/drive/MyDrive/cse6242/{section}')

        if path.exists(f'/content/drive/MyDrive/cse6242/{section}/{station_id}') == False:
            os.mkdir(f'/content/drive/MyDrive/cse6242/{section}/{station_id}')

        keyword = important_section_names[section]

        #https://www.ndbc.noaa.gov/view_text_file.php?filename=0y2w3h2013.txt.gz&dir=data/historical/stdmet/
        link =  f"https://www.ndbc.noaa.gov/view_text_file.php?filename={station_id.lower()}{keyword}{year}.txt.gz&dir=data/historical/{section}/"

        try:
          data = pd.read_csv(link, delim_whitespace=True, usecols=selected_columns, error_bad_lines=False)
          save_path = f"/content/drive/MyDrive/cse6242/{section}/{station_id}/{year}.csv"
          data.to_csv(save_path, index=False)
          success_stations.append(station_id)
        except pd.errors.EmptyDataError:
          # Handle the case where the link contains no data
          print(f"No data found in the provided link for station{station_id} at year {year}")
          no_data_stations.append(station_id)

  else:
    failed_year_ocean.append(station_id)




In [ ]:
len(success_stations)

788

# 11-30 Getting All stdmet Files

In [ ]:
important_section_names = { "ocean": "o", "stdmet": "h"}
stdmet_vals = [ "MM", "DD", "hh", "WVHT", "DPD"] #significant wave height, dominant wave period

In [ ]:
#Code for looping over all "sections" 11-30
no_data_stations = []
for station_id in sections[1]: #loop through every station in the standard meteorlogical section
    if station_id not in failed_stations:
      for download_link in sections[1][station_id]:
            match = re.search(r'h(\d+)\.txt', download_link) #find year in link
            section = download_link.rsplit('/', 2)[-2]
            keyword = sections_name[section]
            # Extract the matched elements

            if match:
                year = match.group(1)#get year

                #create dir if doesnt exist already
                if path.exists(f'/content/drive/MyDrive/cse6242/wave_data/11-30/{section}') == False:
                    os.mkdir(f'/content/drive/MyDrive/cse6242/wave_data/11-30/{section}')

                if path.exists(f'/content/drive/MyDrive/cse6242/wave_data/11-30/{section}/{station_id}') == False:
                    os.mkdir(f'/content/drive/MyDrive/cse6242/wave_data/11-30/{section}/{station_id}')


                    #https://www.ndbc.noaa.gov/view_text_file.php?filename=0y2w3h2013.txt.gz&dir=data/historical/stdmet/
                link_stdmet =  f"https://www.ndbc.noaa.gov/view_text_file.php?filename={station_id.lower()}{keyword}{year}.txt.gz&dir=data/historical/{section}/"
                #print(link)
                try:
                    # get standard meteological data
                    data_stdmet = pd.read_csv(link_stdmet, delim_whitespace=True, usecols=stdmet_vals, error_bad_lines=False)
                    save_path = f"/content/drive/MyDrive/cse6242/wave_data/11-30/{section}/{station_id}/{year}.csv"
                    data_stdmet.to_csv(save_path, index=False)

                except pd.errors.EmptyDataError:
                    # Handle the case where the link contains no data
                    print(f"No data found in the provided link for station{station_id} at year {year}")
                    no_data_stations.append(station_id)


# Get Wave data in statistics for each station
-goal
  - for each station, read in each year, get statistics on the
    - avg, mode, of wave period and wave height
    - store these in the row for the station + coordinates

In [ ]:
import os
import glob
import pandas as pd

In [ ]:
folder_path = '/content/drive/MyDrive/cse6242/wave_data/11-30/'

#stdmet
section = "stdmet"
parent_folder_path = f'{folder_path}/{section}'
all_items = os.listdir(parent_folder_path)
folder_names = [item for item in all_items if os.path.isdir(os.path.join(parent_folder_path, item))]
print(folder_names)

['0Y2W3', '23020', '32301', '32302', '32487', '41003', '41005', '41016', '41018', '41021', '41027', '41030', '41109', '41110', '41113', '41114', '41115', '41117', '41140', '41159', '41193', '42005', '42008', '42009', '42011', '42014', '42015', '42016', '42025', '42089', '42097', '42098', '42099', '42361', '42362', '42363', '42364', '42369', '42370', '42374', '42375', '42376', '42390', '42392', '42394', '42887', '44001', '44006', '44010', '44012', '44022', '44028', '44031', '44035', '44038', '44039', '44040', '44052', '44053', '44056', '44060', '44067', '44073', '44086', '44087', '44089', '44090', '44091', '44093', '44095', '44096', '44097', '44098', '44099', '44100', '45010', '45016', '45020', '45169', '45171', '45172', '45173', '45174', '45180', '46004', '46007', '46008', '46009', '46016', '46017', '46018', '46019', '46020', '46021', '46024', '46039', '46040', '46044', '46048', '46051', '46091', '46092', '46093', '46111', '46114', '46118', '46211', '46212', '46213', '46214', '46215', 

In [ ]:
#import coordinates file
coord_path = '/content/drive/MyDrive/cse6242/station_coord.csv'
coordinates = pd.read_csv(coord_path)
coordinates.columns = ['station_id', 'coordinates']
coordinates

,station_id,coordinates
0,EB31,"{0: '27.000N', 1: '86.000W'}"
1,MEYC1,"{0: '36.605N', 1: '121.889W'}"
2,EGKF1,"{0: '27.601N', 1: '82.759W'}"
3,46255,"{0: '33.400N', 1: '119.651W'}"
4,EMAT2,"{0: '28.710N', 1: '95.914W'}"
...,...,...
1367,45199,"{0: '42.702N', 1: '87.647W'}"
1368,PTJN6,"{0: '40.948N', 1: '73.070W'}"
1369,46116,"{0: '46.287N', 1: '124.016W'}"
1370,EB01,"{0: '36.500N', 1: '73.500W'}"


In [ ]:
###########################
# example file processing #
###########################

#for station 32ST1 or folder_names[0]
combined_stats = pd.DataFrame()
i = 1
station_id = folder_names[i]
path = f'{folder_path}{section}/{folder_names[i]}'
# Use glob to get a list of all CSV files by year in the folder
csv_files = glob.glob(os.path.join(path, '*.csv'))

years = os.listdir(path)
print(csv_files)
empty_dfs = []
i = 0
for csv_file in csv_files:
    df = pd.read_csv(csv_file, dtype={col: float for col in ["WVHT", "DPD"]},skiprows=[1])
    if section == "stdmet":

      #take care of Null entries
      df = df[(df['WVHT'] != 99.0) & (df['DPD'] != 99.0)]

      if df.empty:
        empty_dfs.append(folder_names[i]+years) #collect null stations
      else:

        print(station_id)
        year = int(years[i][:4])
        print(year)
        i += 1
        #update df and save
        # P=0.5×Density of Seawater ×g × H^2 x T
        # P = kg/m^3 * m/s^2 (m^2) * sec
        sea_density = 1020 #km/m^3
        g = 9.81
        df['Power'] = 0.5* 1020 * g* (df['WVHT']** 2) * df['DPD']
        #add the current df to the big df
        # columns: station_id, coordinates, year, "MM", "DD", "hh", power of the year
        df['station_id'] = station_id
        df['YYYY'] = year
        df['coord'] = coordinates.loc[coordinates['station_id'] == station_id, 'coordinates'].values[0]
        df_to_concat = df.copy().drop(columns= [ "WVHT", "DPD"])

        desired_order = ['station_id', 'coord', 'YYYY', 'MM', 'DD', 'hh', 'Power']

        # Rearrange the columns in the DataFrame
        df_to_concat = df_to_concat[desired_order]

        #save the updated file for future use (?)
        #data_stdmet.to_csv(csv_file, index=False)
        combined_stats = pd.concat([combined_stats, df_to_concat], ignore_index=True)

        print(combined_stats)
print(empty_dfs)

['/content/drive/MyDrive/cse6242/wave_data/11-30/stdmet/23020/2008.csv', '/content/drive/MyDrive/cse6242/wave_data/11-30/stdmet/23020/2009.csv', '/content/drive/MyDrive/cse6242/wave_data/11-30/stdmet/23020/2010.csv']
23020
2008
     station_id                         coord  YYYY  MM  DD  hh         Power
0         23020  {0: '22.162N', 1: '38.500E'}  2008  10   9  12   2331.044352
1         23020  {0: '22.162N', 1: '38.500E'}  2008  10   9  13   2048.769450
2         23020  {0: '22.162N', 1: '38.500E'}  2008  10   9  14   3622.224388
3         23020  {0: '22.162N', 1: '38.500E'}  2008  10   9  15   5028.590794
4         23020  {0: '22.162N', 1: '38.500E'}  2008  10   9  16   7468.307482
...         ...                           ...   ...  ..  ..  ..           ...
1981      23020  {0: '22.162N', 1: '38.500E'}  2008  12  31  18  18518.354266
1982      23020  {0: '22.162N', 1: '38.500E'}  2008  12  31  19  21641.534438
1983      23020  {0: '22.162N', 1: '38.500E'}  2008  12  31  20  18771

In [ ]:
###########################
# actual file processing #
###########################
folder_path = '/content/drive/MyDrive/cse6242/wave_data/11-30/'
combined_wave_data = pd.DataFrame()

for idx in range(len(folder_names)):
  station_id = folder_names[idx]

  path = f'{folder_path}{section}/{station_id}'
  # Use glob to get a list of all CSV files by year in the folder
  csv_files = glob.glob(os.path.join(path, '*.csv'))

  years = os.listdir(path)
  empty_dfs = []

  i = 0 #for getting year in years
  #get each year.csv
  for csv_file in csv_files:
      df = pd.read_csv(csv_file, dtype={col: float for col in ["WVHT", "DPD"]},skiprows=[1])
      if section == "stdmet":

        #take care of Null entries
        df = df[(df['WVHT'] != 99.0) & (df['DPD'] != 99.0)]
        year = int(years[i][:4])
        i += 1
        if df.empty:
          empty_dfs.append(station_id+str(year)) #collect null stations

        else:

          #update df and save
          # P=0.5×Density of Seawater ×g × H^2 x T
          # P = kg/m^3 * m/s^2 (m^2) * sec
          sea_density = 1020 #km/m^3
          g = 9.81
          df['Power'] = 0.5* 1020 * g* (df['WVHT']** 2) * df['DPD']
          #add the current df to the big df
          # columns: station_id, coordinates, year, "MM", "DD", "hh", power of the year
          df['station_id'] = station_id
          df['YYYY'] = year
          try:
            df['coord'] = coordinates.loc[coordinates['station_id'] == station_id, 'coordinates'].values[0]
          except:
            df['coord'] = "Error"
          df_to_concat = df.copy().drop(columns= [ "WVHT", "DPD"])

          desired_order = ['station_id', 'coord', 'YYYY', 'MM', 'DD', 'hh', 'Power']

          # Rearrange the columns in the DataFrame
          df_to_concat = df_to_concat[desired_order]

          #save the updated file for future use (?)
          #data_stdmet.to_csv(csv_file, index=False)
          combined_wave_data = pd.concat([combined_wave_data, df_to_concat], ignore_index=True)

combined_wave_data_save_path = '/content/drive/MyDrive/cse6242/combined_wave_data.csv'
combined_wave_data.to_csv(combined_wave_data_save_path, index=False)
print(empty_dfs)
print(combined_stats)

<ipython-input-102-96f10a10f232>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Power'] = 0.5* 1020 * g* (df['WVHT']** 2) * df['DPD']
<ipython-input-102-96f10a10f232>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['station_id'] = station_id
<ipython-input-102-96f10a10f232>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

['YKTV22020', 'YKTV22021', 'YKTV22022']
      station_id                         coord  YYYY  MM  DD  hh        Power
0          23020  {0: '22.162N', 1: '38.500E'}  2008  10   9  12  2331.044352
1          23020  {0: '22.162N', 1: '38.500E'}  2008  10   9  13  2048.769450
2          23020  {0: '22.162N', 1: '38.500E'}  2008  10   9  14  3622.224388
3          23020  {0: '22.162N', 1: '38.500E'}  2008  10   9  15  5028.590794
4          23020  {0: '22.162N', 1: '38.500E'}  2008  10   9  16  7468.307482
...          ...                           ...   ...  ..  ..  ..          ...
18341      23020  {0: '22.162N', 1: '38.500E'}  2010  12  16  23  1830.309088
18342      23020  {0: '22.162N', 1: '38.500E'}  2010  12  17   0  1914.461231
18343      23020  {0: '22.162N', 1: '38.500E'}  2010  12  17   1  2103.803550
18344      23020  {0: '22.162N', 1: '38.500E'}  2010  12  17   2  1659.503255
18345      23020  {0: '22.162N', 1: '38.500E'}  2010  12  17   3  1410.333865

[18346 rows x 7 columns

In [ ]:
combined_wave_data.shape

(5000393, 7)

### Get AVG Power by Year

In [ ]:
###########################
# actual file processing #
###########################
folder_path = '/content/drive/MyDrive/cse6242/wave_data/11-30/'
avg_by_year_wave_data = pd.DataFrame()

for idx in range(len(folder_names)):
  station_id = folder_names[idx]

  path = f'{folder_path}{section}/{station_id}'
  # Use glob to get a list of all CSV files by year in the folder
  csv_files = glob.glob(os.path.join(path, '*.csv'))

  years = os.listdir(path)
  empty_dfs = []

  i = 0 #for getting year in years
  #get each year.csv
  for csv_file in csv_files:
      df = pd.read_csv(csv_file, dtype={col: float for col in ["WVHT", "DPD"]},skiprows=[1])
      if section == "stdmet":

        #take care of Null entries
        df = df[(df['WVHT'] != 99.0) & (df['DPD'] != 99.0)]

        if df.empty:
          empty_dfs.append(station_id+str(year)) #collect null stations

        else:
          year = int(years[i][:4])
          i += 1
          #update df and save
          # P=0.5×Density of Seawater ×g × H^2 x T
          # P = kg/m^3 * m/s^2 (m^2) * sec
          sea_density = 1020 #km/m^3
          g = 9.81
          df['Power'] = 0.5* 1020 * g* (df['WVHT']** 2) * df['DPD']
          #add the current df to the big df
          # columns: station_id, coordinates, year, "MM", "DD", "hh", power of the year
          average_power = df['Power'].mean()
          try:
            coord = coordinates.loc[coordinates['station_id'] == station_id, 'coordinates'].values[0]
          except:
            coord = "Error"
          # New row data
          new_row = {'station_id': station_id, 'coord': coord, 'YYYY': year, 'avg_power':  average_power}

          # Append the new row
          avg_by_year_wave_data = avg_by_year_wave_data.append(new_row, ignore_index=True)


avg_by_year_wave_data_path = '/content/drive/MyDrive/cse6242/avg_by_year_wave_data.csv'
avg_by_year_wave_data.to_csv(avg_by_year_wave_data_path, index=False)
print(empty_dfs)
print(avg_by_year_wave_data_path)

<ipython-input-100-e402aefda221>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  avg_by_year_wave_data = avg_by_year_wave_data.append(new_row, ignore_index=True)
<ipython-input-100-e402aefda221>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  avg_by_year_wave_data = avg_by_year_wave_data.append(new_row, ignore_index=True)
<ipython-input-100-e402aefda221>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  avg_by_year_wave_data = avg_by_year_wave_data.append(new_row, ignore_index=True)
<ipython-input-100-e402aefda221>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  avg_by_year_wave_data = avg_by_year_wave_data.append(new_row, ignore_

['YKTV22021', 'YKTV22021', 'YKTV22021']
/content/drive/MyDrive/cse6242/avg_by_year_wave_data.csv


# get AVG Power by Season and Month

In [ ]:
###########################
# actual file processing #
###########################
folder_path = '/content/drive/MyDrive/cse6242/wave_data/11-30/'
seasonal_avg = pd.DataFrame()
monthly_avg = pd.DataFrame()

#for idx in range(2): for testing
for idx in range(len(folder_names)):
  station_id = folder_names[idx]

  path = f'{folder_path}{section}/{station_id}'
  # Use glob to get a list of all CSV files by year in the folder
  csv_files = glob.glob(os.path.join(path, '*.csv'))

  years = os.listdir(path)
  empty_dfs = []

  i = 0 #for getting year in years
  #get each year.csv
  for csv_file in csv_files:
      df = pd.read_csv(csv_file, dtype={col: float for col in ["WVHT", "DPD"]},skiprows=[1])
      if section == "stdmet":

        #take care of Null entries
        df = df[(df['WVHT'] != 99.0) & (df['DPD'] != 99.0)]
        year = int(years[i][:4])
        i += 1
        if df.empty:
          empty_dfs.append(station_id+str(year)) #collect null stations

        else:

          #update df and save
          # P=0.5×Density of Seawater ×g × H^2 x T
          # P = kg/m^3 * m/s^2 (m^2) * sec
          sea_density = 1020 #km/m^3
          g = 9.81
          df['Power'] = 0.5* 1020 * g* (df['WVHT']** 2) * df['DPD']
          #add the current df to the big df
          # columns: station_id, coordinates, year, "MM", "DD", "hh", power of the year

          # Create a new column for the month and another for the season
          df['season'] = (df['MM'] % 12 + 3) // 3
          df = df.drop(columns= [ "WVHT", "DPD"])

          # Calculate monthly averages
          monthly_averages = df.groupby('MM')['Power'].mean().reset_index()

          # Calculate seasonal averages
          seasonal_averages = df.groupby('season')['Power'].mean().reset_index()
          try:
            coord = coordinates.loc[coordinates['station_id'] == station_id, 'coordinates'].values[0]
          except:
            coord = "Error"

          seasonal_averages['station_id'] = station_id
          seasonal_averages['YYYY'] = year
          seasonal_averages['coord'] = coord
          #seasonal_averages = seasonal_averages.drop(columns= [ "WVHT", "DPD"])

          monthly_averages['station_id'] = station_id
          monthly_averages['YYYY'] = year
          monthly_averages['coord'] = coord

          # Rearrange the columns in the DataFrame
          #df_to_concat = df_to_concat[desired_order]

          #save the updated file for future use (?)
          #data_stdmet.to_csv(csv_file, index=False)
          seasonal_avg = pd.concat([seasonal_avg, seasonal_averages], ignore_index=True)
          monthly_avg = pd.concat([monthly_avg, monthly_averages], ignore_index=True)

desired_order = ['station_id', 'coord', 'YYYY', 'season', 'Power']
seasonal_avg = seasonal_avg[desired_order]


seasonal_avg_path = '/content/drive/MyDrive/cse6242/seasonal_avg.csv'
monthly_avg_path = '/content/drive/MyDrive/cse6242/monthly_avg.csv'

seasonal_avg.to_csv(seasonal_avg_path, index=False)
monthly_avg.to_csv(monthly_avg_path, index=False)

print(empty_dfs)
print(seasonal_avg)
print(monthly_avg)

<ipython-input-19-d0710564c88c>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Power'] = 0.5* 1020 * g* (df['WVHT']** 2) * df['DPD']
<ipython-input-19-d0710564c88c>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['season'] = (df['MM'] % 12 + 3) // 3
<ipython-input-19-d0710564c88c>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

['YKTV22020', 'YKTV22021', 'YKTV22022']
     station_id                          coord  YYYY  season         Power
0         23020   {0: '22.162N', 1: '38.500E'}  2008       1  34099.362781
1         23020   {0: '22.162N', 1: '38.500E'}  2008       4  32434.691435
2         23020   {0: '22.162N', 1: '38.500E'}  2010       1  25743.558717
3         23020   {0: '22.162N', 1: '38.500E'}  2010       2  47231.448798
4         23020   {0: '22.162N', 1: '38.500E'}  2010       3  36822.310721
...         ...                            ...   ...     ...           ...
1439      VBBA3  {0: '36.132N', 1: '114.412W'}  2020       2    376.448287
1440      VBBA3  {0: '36.132N', 1: '114.412W'}  2020       3    186.526402
1441      VBBA3  {0: '36.132N', 1: '114.412W'}  2020       4    658.109514
1442      VBBA3  {0: '36.132N', 1: '114.412W'}  2021       1    585.052153
1443      VBBA3  {0: '36.132N', 1: '114.412W'}  2021       2    140.038198

[1444 rows x 5 columns]
      MM         Power station_id  

In [ ]:
seasonal_avg.to_csv(seasonal_avg_path, index=False)


In [ ]:
desired_order = ['station_id', 'coord', 'YYYY', 'MM', 'Power']
monthly_avg = monthly_avg[desired_order]
monthly_avg_path = '/content/drive/MyDrive/cse6242/monthly_avg.csv'
monthly_avg.to_csv(monthly_avg_path, index=False)

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/cse6242/combined_wave_data.csv'
data = pd.read_csv(file_path)
data.shape


<ipython-input-2-3a5dbf19de63>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


(5000393, 7)